In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
import util
from sklearn.multioutput import MultiOutputRegressor
from numpy import linalg as LA
from baseline import MeanRegressor
import blender_diff

VICON_FREQ = 30

In [2]:
def true_line_pred_scatter(y_test, y_pred, yd, title):
    """
    Plot all test data as line plot and prediction as scatter
    :param y_test: test data
    :param y_pred: prediction data
    :param yd: dimension index of y
    :param title: plot title
    """
    label = 'y_' + str(yd + 1)
    label_true = label + ' true'
    label_pred = label + ' predicted'
    shape = y_test.shape
    plt.plot(np.arange(shape[0]) / 30, y_test[:, yd], label=label_true)
    plt.scatter(np.arange(shape[0])/30, y_pred, 8, 'r', alpha=0.7, label=label_pred)
    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [4]:
def plot_rmse_series(rmse, names):
    """
    Plot RMSE values in time series
    :param rmse: numpy array of size n_models x n_rows
    """
    count = 100
    for yd in range(rmse.shape[-1]):
        for i, name in enumerate(names):
            plt.plot(np.arange(count) / VICON_FREQ, rmse[i, :count, yd])
        plt.legend(names)
        plt.show()

In [3]:
XFILE = 'data/imu_proc01-Feb-2018.csv'
YFILE = 'data/vicon_proc01-Feb-2018.csv'
X = np.genfromtxt(XFILE, delimiter=',')
X = X[:, 10:]
y = np.genfromtxt(YFILE, delimiter=',')
perm = [1, 2, 0, 4, 3]  # permutation of cols
y = y[:, perm]
y[:, 3:5] = -y[:, 3:5]

In [4]:
# Split train and test set
X_train, X_test, y_train, y_test = util.test_split(X, y, 1000, 0.1)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_test_idx = y_test[:, -1]

In [5]:
mlp = MLPRegressor(500, max_iter=1000, alpha=0.1)
svr = SVR(C=10, gamma=0.1, epsilon=1)
svr = MultiOutputRegressor(svr, n_jobs=-1)
mean = MeanRegressor()
models = [svr, mlp, mean]

In [6]:
names = ['SVM', 'MLP', 'Mean']
yd_names = ['Shoulder x', 'Shoulder y', 'Shoulder z', 'Elbow y', 'Elbow z']

In [8]:
for model in models:
    model.fit(X_train, y_train)

In [9]:
y_pred = np.zeros((len(models), y_test.shape[0], y_test.shape[1]))
for i, model in enumerate(models):
    y_pred[i] = util.moving_avg_smooth(model.predict(X_test))

In [10]:
rmse = np.zeros((len(models), y_test.shape[-1]))
for i, model in enumerate(models):
    rmse[i] = mean_squared_error(y_test, y_pred[i], multioutput='raw_values')
rmse = np.sqrt(rmse)

In [13]:
# Plot bar graphs showing RMSE mean and std
N = 5
ind = np.arange(N)  # the x locations for the groups
width = 0.25       # the width of the bars
colors = ['red', 'green', 'black']
fig, ax = plt.subplots()
rects = []
for i in range(rmse.shape[0]):
    rects.append(ax.bar(ind + i*width, rmse[i], width, color=colors[i]))

# add some text for labels, title and axes ticks
ax.set_ylabel('RMSE (deg)')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(yd_names)
ax.legend(rects, names)
plt.savefig('results/rmse.png')
plt.show()

In [11]:
r2 = np.zeros_like(rmse)
for i, model in enumerate(models[:2]):
    r2[i] = r2_score(y_test, y_pred[i], multioutput='raw_values')

In [12]:
# Plot bar graphs showing R2 scores
N = 5
ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars
colors = ['red', 'green']
fig, ax = plt.subplots()
rects = []
for i in range(r2.shape[0]-1):
    rects.append(ax.bar(ind + i*width, r2[i], width, color=colors[i]))

# add some text for labels, title and axes ticks
ax.set_ylabel('R2 score')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(yd_names)
ax.legend(rects, names[:2])
plt.savefig('results/r2.png')
plt.show()


In [10]:
# plot true vs prediction scatter
n_models = len(models) - 1
n_data = y_test.shape[1]
fig, ax = plt.subplots(n_data, n_models)
for m in range(n_models):
    for i in range(y_test.shape[1]):
        label = yd_names[i]
        label_true = label + ' true'
        label_pred = label + ' predicted'
        y_true = y_test[:, i]
        pred = y_pred[m, :, i]
        high = max(pred.max(), y_true.max())
        low = min(pred.min(), y_true.min())
        ax[i, m].plot([low, high], [low, high])
        ax[i, m].scatter(y_true, pred, 2)
        ax[i, m].set_xlabel(label_true)
        if m == 0:
            ax[i, m].set_ylabel(label_pred)
        if i == 0:
            ax[i, m].set_title(names[m])
fig.set_figheight(15)
fig.show()

In [11]:
# plot value distribution
bins = 20
X_hist = np.zeros((X.shape[1], bins))
y_hist = np.zeros((y.shape[1], bins))
for i in range(X.shape[1]):
    X_hist[i], _ = np.histogram(X[:, i], bins)
for i in range(y.shape[1]):
    y_hist[i], _ = np.histogram(y[:, i], bins)

fig, ax = plt.subplots(3, 2)
X_min = X.min(0)
X_max = X.max(0)
y_min = y.min(0)
y_max = y.max(0)
# angles
for i in range(4):
    ax[0, 0].plot(np.linspace(X_min[i], X_max[i], bins), X_hist[i], 'o-')
ax[0, 0].set_xlabel('IMU Wrist Angles')
ax[0, 0].set_ylabel('Frequency')
ax[0, 0].legend((r'$q_w$', r'$q_x$', r'$q_y$', r'$q_z$'))

# angular velocity
for i in range(4, 7):
    ax[1, 0].plot(np.linspace(X_min[i], X_max[i], bins), X_hist[i], 'o-')
ax[1, 0].set_xlabel('IMU Wrist Angular Velocity (deg/s)')
ax[1, 0].set_ylabel('Frequency')
ax[1, 0].legend((r'$\omega_x$', r'$\omega_y$', r'$\omega_z$'))

# angular acceleration
for i in range(7, 10):
    ax[2, 0].plot(np.linspace(X_min[i], X_max[i], bins), X_hist[i], 'o-')
ax[2, 0].set_xlabel('IMU Wrist Angular Acceleration (deg/' + r'$s^2$' + ')')
ax[2, 0].set_ylabel('Frequency')
ax[2, 0].legend((r'$a_x$', r'$a_y$', r'$a_z$'))

# Vicon shoulder
for i in range(3):
    ax[0, 1].plot(np.linspace(y_min[i], y_max[i], bins), y_hist[i], 'o-')
ax[0, 1].set_xlabel('MoCap Shoulder Angles (deg)')
ax[0, 1].legend((r'$\theta_x$', r'$\theta_y$', r'$\theta_z$'))

# Vicon shoulder
for i in range(3, 5):
    ax[1, 1].plot(np.linspace(y_min[i], y_max[i], bins), y_hist[i], 'o-')
ax[1, 1].set_xlabel('MoCap Elbow Angles (deg)')
ax[1, 1].legend((r'$\theta_y$', r'$\theta_z$'))

ax[2, 1].set_visible(False)
fig.set_figwidth(7)
fig.set_figheight(11)
plt.savefig('../journal_imu_tracking/images/distribution.png')
plt.show()